In [2]:
# moduleのインポート
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
import matplotlib.ticker as ticker
from sklearn.cluster import KMeans



In [3]:
# データのパス
data_path = "/Users/miyazakishinichi/Library/CloudStorage/GoogleDrive-s.miyazaki.med@gmail.com/.shortcut-targets-by-id/1ChgdmRlKZ2klXuoWPaMWiX2pLhJ1-0nY/miyazaki/python_analysis/MCRtest/pyMCR/Data.csv"
# 横軸のパス
axis_path = "/Users/miyazakishinichi/Library/CloudStorage/GoogleDrive-s.miyazaki.med@gmail.com/.shortcut-targets-by-id/1ChgdmRlKZ2klXuoWPaMWiX2pLhJ1-0nY/miyazaki/python_analysis/MCRtest/pyMCR/re_ramanshift2.csv"

# PCA後のデータをいくつのクラスターに分けるか
cluster_num = 10
# データの縦幅
x = 501
# データの横幅
y = 351
# 詳細に解析する波数範囲 (指紋領域用にしています)
subrange = (700, 3500)

def normalization(data_array):
    """
    normalize the data array within 0~255
    :param data_array: numpy array
    :return: data_array: numpy array (overwrite)
    """
    amin = np.amin(data_array)
    amax = np.amax(data_array)
    scale = 255.0 / (amax - amin)
    data_array = data_array - amin
    data_array = data_array * scale
    data_array = np.uint8(data_array)
    return data_array

def PCA_analysis(data, x_axis, flag):
    # data normalization
    norm_data = data.iloc[:, 1:].apply(lambda x: (x - x.mean()) / x.std(), axis=0)
    pca = PCA()
    pca.fit(norm_data)
    feature = pca.transform(norm_data)
    data_PCA = pd.DataFrame(feature,
                            columns=["PC{}".format(x + 1) for x in range(len(norm_data.columns))])

    Spectrum = []

    # make graph for PC scatter
    os.makedirs("./results/figures", exist_ok=True)
    plt.figure(figsize=(6, 6))
    plt.scatter(feature[:, 0], feature[:, 1], alpha=0.8, color="black")
    plt.grid()
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.grid(False)
    plt.savefig("./results/figures/PCA_scatter_{}.png".format(flag), transparent=True)

    # make graph for CDF
    plt.figure(figsize=(8, 5))
    pd.DataFrame(pca.explained_variance_ratio_, index=["PC{}".format(x + 1) for x in range(len(norm_data.columns))])
    plt.gca().get_xaxis().set_major_locator(ticker.MaxNLocator(integer=True))
    plt.plot([0] + list(np.cumsum(pca.explained_variance_ratio_)), "-o",
             color="black")
    plt.xlabel("Number of principal components")
    plt.ylabel("Cumulative contribution rate")
    plt.grid()
    plt.xlim(0, 15)
    plt.grid(False)
    plt.savefig("./results/figures/PCA_CDF_{}.png".format(flag), transparent=True, bbox_inches="tight", pad_inches=0.1)

    # clustering and
    plt.figure(figsize=(30, 30))
    extracted_df = data_PCA
    cust_array = extracted_df.to_numpy()
    cust_array = cust_array
    pred = KMeans(n_clusters=cluster_num).fit_predict(cust_array)
    pred_image = np.reshape(pred, [x, y])
    plt.imshow(pred_image, cmap="rainbow")
    plt.savefig('./results/figures/clustered_img_{}.png'.format(flag))
    # plt.imsave('./results/figures/clustered_img_{}.png'.format(flag), pred_image)

    # class differentiation
    os.makedirs("./results/spectrum_{}".format(flag), exist_ok=True)
    os.makedirs("./results/figures/Class_images_{}".format(flag), exist_ok=True)
    Class_list = []
    for i in range(cluster_num):
        Class_list.append(np.where(pred != i, 0, 1))
        plt.imsave("./results/figures/Class_images_{0}/Class{1}.png".format(flag, i),
                   np.reshape(np.where(pred != i, 0, 1), [x, y]))

    for i in range(cluster_num):
        fig, ax = plt.subplots()
        Class_data = data.T * Class_list[i]
        Class_data = np.mean(Class_data.T.values, axis=0)
        Class_min = np.min(Class_data)
        Class_max = np.max(Class_data)
        norm_Class_data = (Class_data - Class_min) / (Class_max - Class_min)
        ax.set_ylim([-0.2, 1])
        ax.grid(False)
        ax.invert_xaxis()
        ax.plot(x_axis, norm_Class_data, color="black")
        plt.savefig("./results/spectrum_{0}/norm_Class_{1}.png".format(flag, i),
                    transparent=True, bbox_inches="tight", pad_inches=0.1)

        fig, ax = plt.subplots()
        ax.set_ylim([-0.2, 1])
        ax.grid(False)
        ax.invert_xaxis()
        ax.plot(x_axis, Class_data, color="black")
        plt.savefig("./results/spectrum_{0}/Class_{1}.png".format(flag, i),
                    transparent=True, bbox_inches="tight", pad_inches=0.1)

        pd.DataFrame(Class_data).to_csv("./results/spectrum_{0}/Class_{1}.csv".format(flag, i))
        Spectrum.append(Class_data)
    return Spectrum

def execute_analysis(data_path, axis_path):
    os.chdir(os.path.dirname(data_path))
    # all range analysis
    data = pd.read_csv(data_path, header=None).T[0:x * y]
    x_axis = np.loadtxt(axis_path)
    Spectrum = PCA_analysis(data, x_axis, flag="full")
    #Entropy_analysis(data, x_axis, flag="full")

    # subrange analysis
    subrange_axis = x_axis[np.where((x_axis > subrange[0]) & (x_axis < subrange[1]))]
    subrange_data = data.T.iloc[np.where((x_axis > subrange[0]) & (x_axis < subrange[1]))].T
    PCA_analysis(subrange_data, subrange_axis, flag="subrange")
    #Entropy_analysis(data, x_axis, flag="subrange")

    return Spectrum, data


Spectrum, data = execute_analysis(data_path, axis_path)

KeyboardInterrupt: 

In [ ]:
Spectrum = np.array(Spectrum)
S = pd.DataFrame(Spectrum)

In [ ]:
#S = S.iloc[:, 1:].apply(lambda x: (x - x.mean()) / x.std(), axis=0)
S.values.shape


In [ ]:
from pymcr.mcr import McrAR
from pymcr.regressors import NNLS, OLS
from pymcr.constraints import ConstraintNonneg, ConstraintNorm

# Note constraint order matters
mcrar = McrAR(c_regr=OLS(), st_regr=OLS(), c_constraints=[ConstraintNonneg()],
              st_constraints=[ConstraintNonneg()])

In [ ]:
D = data
#D = D.iloc[:, 1:].apply(lambda x: (x - x.mean()) / x.std(), axis=0)
D = D.values


In [ ]:

D.shape

In [ ]:
mcrar.fit(D, ST=Spectrum, verbose=True)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(6,10))
plt.subplot(311)
plt.plot(Spectrum.T)
plt.xlabel(r'Wavenumber (cm$^{-1}$)')
plt.ylabel('Intensity (au)')
plt.title('Initial')

plt.subplot(312)
plt.plot(mcrar.ST_opt_.T)
plt.xlabel(r'Wavenumber (cm$^{-1}$)')
plt.ylabel('Intensity (au)')
plt.title('MCR-AR Retrieved')

plt.subplot(313)
plt.plot(Spectrum.T - mcrar.ST_opt_.T)
plt.xlabel(r'Wavenumber (cm$^{-1}$)')
plt.ylabel('Intensity (au)')
plt.title('Difference')

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(4,10))
plt.subplot(251)
plt.imshow(mcrar.C_opt_[:,0].reshape(x,y))
plt.subplot(252)
plt.imshow(mcrar.C_opt_[:,2].reshape(x,y))
plt.subplot(253)
plt.imshow(mcrar.C_opt_[:,1].reshape(x,y))
plt.subplot(254)
plt.imshow(mcrar.C_opt_[:,3].reshape(x,y))
plt.subplot(255)
plt.imshow(mcrar.C_opt_[:,4].reshape(x,y))
plt.subplot(256)
plt.imshow(mcrar.C_opt_[:,5].reshape(x,y))
plt.subplot(257)
plt.imshow(mcrar.C_opt_[:,6].reshape(x,y))
plt.subplot(258)
plt.imshow(mcrar.C_opt_[:,7].reshape(x,y))
plt.subplot(259)
plt.imshow(mcrar.C_opt_[:,8].reshape(x,y))
plt.subplot(2510)
plt.imshow(mcrar.C_opt_[:,9].reshape(x,y))
plt.savefig("images.png")

In [ ]:
plt.imshow(mcrar.C_opt_[:,0].reshape(501,351))


In [ ]:
plt.plot(mcrar.ST_opt_.T[:,0])
plt.xlabel(r'Wavenumber (cm$^{-1}$)')
plt.ylabel('Intensity (au)')
plt.title('MCR-AR Retrieved')

In [ ]:
plt.figure(figsize=(10,4))
for i in range(10):
    plt.subplot(5,2,i)
    plt.plot(mcrar.ST_opt_.T[:,i])
    plt.xlabel(r'Wavenumber (cm$^{-1}$)')
    plt.ylabel('Intensity (au)')
plt.savefig("spectrum.png")

In [ ]:
plt.figure(figsize=(10,4))
for i in range(10):
    plt.subplot(5,2,i)
    plt.plot(mcrar.C_opt_[:,i].reshape(x,y))
    plt.tick_params(bottom=False, left=False, right=False, top=False)
    plt.tick_params(labelbottom=False, labelleft=False, labelright=False, labeltop=False)
plt.savefig("image.png")